In [44]:
# Import necessary libraries

In [45]:
import sqlite3
import pandas as pd
from pathlib import Path
import numpy as np

In [46]:
# Defining the path to the database
db_path = Path(r'n:\Projects\11209000\11209353\B. Measurements and calculations\008 - Resultaten Proefvlucht\ZZL\7-2\databases\database_bekleding_bovengrens\traject_7_2.db')

In [47]:
# Connecting to the database
conn = sqlite3.connect(db_path)

In [48]:
# obtain a list of all tables in the database
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)

print("The database consists of the following", len(tables), "tables:")
print(tables)
print()

The database consists of the following 35 tables:
                               name
0                   DikeTrajectInfo
1                       SectionData
2                         Mechanism
3               MechanismPerSection
4         AssessmentMechanismResult
5           AssessmentSectionResult
6                   ComputationType
7               ComputationScenario
8                         SlopePart
9            BlockRevetmentRelation
10                        Buildings
11          CharacteristicPointType
12                   CombinableType
13     ComputationScenarioParameter
14                      MeasureType
15                          Measure
16                    CustomMeasure
17           CustomMeasureParameter
18           GrassRevetmentRelation
19                MeasurePerSection
20                    MeasureResult
21           MeasureResultMechanism
22           MeasureResultParameter
23             MeasureResultSection
24                   MechanismTable
25            

In [49]:
# We want to receive an overview of which measures are taken for the (VRM) or the Doorsnede-Eisen
# Methode (DEM)

In [50]:
# Defining an SQL query
table_name_id = 26 # this is the optimization run
sql_query = 'SELECT * FROM {}'.format(tables.iloc[table_name_id].values[0])

In [51]:
# Retrieving the data from the database and write the optimization_type name and 
# optimization_type_id to a dictionary:
optimization_type = pd.read_sql_query(sql_query, conn)
optimization_type_dict = dict(zip(optimization_type["name"], optimization_type["optimization_type_id"]))
print(optimization_type_dict)

{'Basisberekening Veiligheidsrendement': 1, 'Basisberekening Doorsnede-eisen': 2}


In [52]:
# Retrieve from the database tab "OptimizationSelectedMeasure" the id and investment_year 
# for all the measures for optimization_run_id = 1.
optimization_run_id = 1 # 1 is VRM, 2 is DSM
sql_query = 'SELECT * FROM OptimizationSelectedMeasure WHERE optimization_run_id = {}'.format(optimization_run_id)
selected_optimization_measure = pd.read_sql_query(sql_query, conn)
print(selected_optimization_measure)

          id  optimization_run_id  measure_result_id  investment_year
0          1                    1                  1                0
1          2                    1                  1               20
2          3                    1                  2                0
3          4                    1                  2               20
4          5                    1                  3                0
...      ...                  ...                ...              ...
12116  12117                    1               7077                0
12117  12118                    1               7078                0
12118  12119                    1               7079                0
12119  12120                    1               7080                0
12120  12121                    1               7081                0

[12121 rows x 4 columns]


In [53]:
# create a list of ids in the selected_optimization_measure
selected_measure_ids = selected_optimization_measure["id"].tolist()

In [54]:
# retrieve from the database tab "OptimizationStep" all rows where optimization_selected_measure_id is in selected_measure_ids
sql_query = 'SELECT * FROM OptimizationStep WHERE optimization_selected_measure_id IN ({})'.format(
    ', '.join([str(i) for i in selected_measure_ids]))
optimization_step = pd.read_sql_query(sql_query, conn)
# sort optimizationStep by id
optimization_step = optimization_step.sort_values(by="id")
print(optimization_step)

      id  optimization_selected_measure_id  step_number     total_lcc  \
7      1                               399            1  8.248762e+04   
67     2                              3148            2  5.096304e+05   
79     3                              3467            3  1.435300e+06   
5      4                               355            4  1.476544e+06   
66     5                              3143            5  1.690116e+06   
..   ...                               ...          ...           ...   
14   467                               703          295  2.426556e+08   
241  468                              6411          296  2.428164e+08   
256  469                              6696          296  2.428164e+08   
93   470                              3577          297  2.437955e+08   
105  471                              3873          297  2.437955e+08   

       total_risk  
7    5.078556e+11  
67   4.795481e+11  
79   4.413895e+11  
5    4.138515e+11  
66   3.944441e+11  
.. 

In [55]:
# retrieve from the database tab "OptimizationStep" all rows where optimization_selected_measure_id is in selected_measure_ids
sql_query = 'SELECT * FROM OptimizationStep WHERE optimization_selected_measure_id IN ({})'.format(
    ', '.join([str(i) for i in selected_measure_ids]))
optimization_step = pd.read_sql_query(sql_query, conn)
# sort optimizationStep by id
optimization_step = optimization_step.sort_values(by="id")
print(optimization_step)

sql_query = 'SELECT * FROM OptimizationSelectedMeasure WHERE id IN ({})'.format(
    ', '.join([str(i) for i in optimization_step["optimization_selected_measure_id"].tolist()]))
optimization_selected_measure = pd.read_sql_query(sql_query, conn)

      id  optimization_selected_measure_id  step_number     total_lcc  \
7      1                               399            1  8.248762e+04   
67     2                              3148            2  5.096304e+05   
79     3                              3467            3  1.435300e+06   
5      4                               355            4  1.476544e+06   
66     5                              3143            5  1.690116e+06   
..   ...                               ...          ...           ...   
14   467                               703          295  2.426556e+08   
241  468                              6411          296  2.428164e+08   
256  469                              6696          296  2.428164e+08   
93   470                              3577          297  2.437955e+08   
105  471                              3873          297  2.437955e+08   

       total_risk  
7    5.078556e+11  
67   4.795481e+11  
79   4.413895e+11  
5    4.138515e+11  
66   3.944441e+11  
.. 

In [56]:
print(optimization_selected_measure)

        id  optimization_run_id  measure_result_id  investment_year
0        3                    1                  2                0
1      312                    1                168                0
2      314                    1                170                0
3      315                    1                171                0
4      317                    1                173                0
..     ...                  ...                ...              ...
317  11803                    1               6889                0
318  11805                    1               6890                0
319  11821                    1               6898                0
320  12058                    1               7018                0
321  12106                    1               7066                0

[322 rows x 4 columns]


In [57]:
# add the measure_result_id + investment_year to the optimization_step dataframe. These can be found in the 
# optimization_selected_measure dataframe
optimization_step = pd.merge(optimization_step, optimization_selected_measure[["id", "measure_result_id", "investment_year"]],
                                left_on="optimization_selected_measure_id", right_on="id", how="left")
# drop the "id_y" column
optimization_step = optimization_step.drop(columns="id_y")
optimization_step = optimization_step.rename(columns={"id_x": "id"})
print(optimization_step)

      id  optimization_selected_measure_id  step_number     total_lcc  \
0      1                               399            1  8.248762e+04   
1      2                              3148            2  5.096304e+05   
2      3                              3467            3  1.435300e+06   
3      4                               355            4  1.476544e+06   
4      5                              3143            5  1.690116e+06   
..   ...                               ...          ...           ...   
466  467                               703          295  2.426556e+08   
467  468                              6411          296  2.428164e+08   
468  469                              6696          296  2.428164e+08   
469  470                              3577          297  2.437955e+08   
470  471                              3873          297  2.437955e+08   

       total_risk  measure_result_id  investment_year  
0    5.078556e+11                255                0  
1    4.7954

In [58]:
# print how many investments are done in year t = 0 and t = 20
print("investments done in year t=0:")
print(np.sum(optimization_step.investment_year==0))
print("investments done in year t=20:")
print(np.sum(optimization_step.investment_year==20))      

investments done in year t=0:
448
investments done in year t=20:
23


In [59]:
# add MeasureResult where id matches optimization_step["measure_result_id"] to the optimization_step dataframe
sql_query = 'SELECT * FROM MeasureResult WHERE id IN ({})'.format(
    ', '.join([str(i) for i in optimization_step["measure_result_id"].tolist()]))
measure_result = pd.read_sql_query(sql_query, conn)
# add the "name" column from the measure_result dataframe to the optimization_step dataframe
optimization_step = pd.merge(optimization_step, measure_result[["id", "measure_per_section_id"]],
                                left_on="measure_result_id", right_on="id", how="left")
# drop the "id_y" column
optimization_step = optimization_step.drop(columns="id_y")
optimization_step = optimization_step.rename(columns={"id_x": "id"})

In [61]:
print(optimization_step)

      id  optimization_selected_measure_id  step_number     total_lcc  \
0      1                               399            1  8.248762e+04   
1      2                              3148            2  5.096304e+05   
2      3                              3467            3  1.435300e+06   
3      4                               355            4  1.476544e+06   
4      5                              3143            5  1.690116e+06   
..   ...                               ...          ...           ...   
466  467                               703          295  2.426556e+08   
467  468                              6411          296  2.428164e+08   
468  469                              6696          296  2.428164e+08   
469  470                              3577          297  2.437955e+08   
470  471                              3873          297  2.437955e+08   

       total_risk  measure_result_id  investment_year  measure_per_section_id  
0    5.078556e+11                255       

In [65]:
# add MeasurePerSection where id matches optimization_step["measure_per_section_id"] to the optimization_step dataframe
sql_query = 'SELECT * FROM MeasurePerSection WHERE id IN ({})'.format(
    ', '.join([str(i) for i in optimization_step["measure_per_section_id"].tolist()]))
measure_per_section = pd.read_sql_query(sql_query, conn)
# add the "section_id", "measure_id" column from the measure_per_section dataframe to the optimization_step dataframe
optimization_step = pd.merge(optimization_step, measure_per_section[["id", "section_id", "measure_id"]],
                                left_on="measure_per_section_id", right_on="id", how="left")
# drop the "id_y" column
optimization_step = optimization_step.drop(columns="id_y")
optimization_step = optimization_step.rename(columns={"id_x": "id"})
print(optimization_step)

      id  optimization_selected_measure_id  step_number     total_lcc  \
0      1                               399            1  8.248762e+04   
1      2                              3148            2  5.096304e+05   
2      3                              3467            3  1.435300e+06   
3      4                               355            4  1.476544e+06   
4      5                              3143            5  1.690116e+06   
..   ...                               ...          ...           ...   
466  467                               703          295  2.426556e+08   
467  468                              6411          296  2.428164e+08   
468  469                              6696          296  2.428164e+08   
469  470                              3577          297  2.437955e+08   
470  471                              3873          297  2.437955e+08   

       total_risk  measure_result_id  investment_year  measure_per_section_id  \
0    5.078556e+11                255      

In [66]:
# Where id in SectionData matches section_id in optimization_step dataframe, add the "section_name" column to the
# optimization_step dataframe
sql_query = 'SELECT * FROM SectionData WHERE id IN ({})'.format(
    ', '.join([str(i) for i in optimization_step["section_id"].tolist()]))
section_data = pd.read_sql_query(sql_query, conn)
# add the "section_name" column from the section_data dataframe to the optimization_step dataframe
optimization_step = pd.merge(optimization_step, section_data[["id", "section_name"]],
                                left_on="section_id", right_on="id", how="left")
# drop the "id_y" column
optimization_step = optimization_step.drop(columns="id_y")
# rename the "id_x" column to "id"
optimization_step = optimization_step.rename(columns={"id_x": "id"})

In [67]:
print(optimization_step)

      id  optimization_selected_measure_id  step_number     total_lcc  \
0      1                               399            1  8.248762e+04   
1      2                              3148            2  5.096304e+05   
2      3                              3467            3  1.435300e+06   
3      4                               355            4  1.476544e+06   
4      5                              3143            5  1.690116e+06   
..   ...                               ...          ...           ...   
466  467                               703          295  2.426556e+08   
467  468                              6411          296  2.428164e+08   
468  469                              6696          296  2.428164e+08   
469  470                              3577          297  2.437955e+08   
470  471                              3873          297  2.437955e+08   

       total_risk  measure_result_id  investment_year  measure_per_section_id  \
0    5.078556e+11                255      

In [ ]:
# Closing the connection to the database
# conn.close()